# welcome to the product uploader!

In [ ]:
!pip install -r requirements.txt

In [ ]:
from api_and_repo_link import link_me_up
import api_and_repo_link
from importlib import reload
reload(api_and_repo_link)
link_me_up()

In [ ]:
from oliverbonas_source_dags.apis.api_netsuite import APIConfig_netsuite
from util.get_ns_number import get_ns_number
from icecream import ic
import xml.etree.ElementTree as ET
import pandas

# Andrei's tables in bq-central-dev

In [ ]:

df = pandas.read_gbq("""
select *
from `bq-central-dev`.data_migration_products.dbt_products__everything_all
limit 1
""")
fashion = pandas.read_gbq("""
select *
from `bq-central-dev`.data_migration_products.netsuite_products__parents_fashion
limit 100
""")
home = pandas.read_gbq("""
select *
from `bq-central-dev`.data_migration_products.netsuite_products__parents_home_and_gift
limit 5
""")
everything = pandas.read_gbq("""
select * 
from `bq-central-dev`.data_migration_products.dbt_products__all_attributes_and_properties
limit 1000
""")


In [ ]:
everything

# Use the API to download data from netsuite, and create the netsuite_mappings

* We use a custom form for InventoryItems, which has customfields
* A customfield has an internalId, a scriptId and a name, if it is a matrix custom field, it may be attached to a custom list
* A customlist has an internalId, a name, and a list of values, Each value in the list has an internalId and a name


- Customfield1 <---> Customlist1 (value1, value2, value3)
- Customfield2 <---> Customlist2 (value1, value2)
- Customfield3 <---> Customlist3 (value1, value2, value3, value4)

#### for the example above, there are 6 + (3 + 2 + 4) = 15 internalIds

In [ ]:
api = APIConfig_netsuite()

In [ ]:


# python library zeep
InventoryItem = api.client.get_type("ns17:InventoryItem")
CostCategory = api.client.get_type("ns17:CostCategory")
CostCategorySearch = api.client.get_type("ns17:CostCategorySearch")
CostCategorySearchBasic = api.client.get_type("ns5:CostCategorySearchBasic")
RecordRef = api.client.get_type("ns0:RecordRef")
StringCustomFieldRef = api.client.get_type(f"ns0:StringCustomFieldRef")
CustomRecordSearchBasic = api.client.get_type("ns5:CustomRecordSearchBasic")
CustomRecordSearch = api.client.get_type("ns32:CustomRecordSearch")
CustomListSearch = api.client.get_type("ns32:CustomListSearch")
CustomListSearchBasic = api.client.get_type("ns5:CustomListSearchBasic")
CustomRecordRef = api.client.get_type("ns0:CustomRecordRef")
AccountSearchBasic = api.client.get_type("ns5:AccountSearchBasic")
SubsidiarySearchBasic = api.client.get_type("ns5:SubsidiarySearchBasic")
SelectCustomFieldRef = api.client.get_type("ns0:SelectCustomFieldRef")
BooleanCustomFieldRef = api.client.get_type("ns0:BooleanCustomFieldRef")
InventoryItemHierarchyVersions = api.client.get_type("ns17:InventoryItemHierarchyVersions")
InventoryItemHierarchyVersionsList = api.client.get_type("ns17:InventoryItemHierarchyVersionsList")
MerchandiseHierarchyNodeSearchBasic = api.client.get_type("ns5:MerchandiseHierarchyNodeSearchBasic")
VendorSearchBasic = api.client.get_type("ns5:VendorSearchBasic")
MultiSelectCustomFieldRef = api.client.get_type("ns0:MultiSelectCustomFieldRef")
ListOrRecordRef = api.client.get_type("ns0:ListOrRecordRef")
DeleteRequest = api.client.get_type("ns4:DeleteRequest")
MerchandiseHierarchyNode = api.client.get_type("ns17:MerchandiseHierarchyNode")
SearchMultiSelectField = api.client.get_type("ns0:SearchMultiSelectField")
DoubleCustomFieldRef = api.client.get_type("ns0:DoubleCustomFieldRef")
LongCustomFieldRef = api.client.get_type("ns0:LongCustomFieldRef")
ItemSearchBasic = api.client.get_type("ns5:ItemSearchBasic")
CustomFieldList = api.client.get_type("ns0:CustomFieldList")
print(get_ns_number(api, "CustomFieldList"))


### 1a. get the custom lists, and custom list values

In [ ]:
response_customLists = api.service_proxy.getCustomizationId(
    customizationType="customList",#"itemCustomField",
    includeInactives=False,
    _soapheaders = api.build_soap_headers(),
)
customLists = response_customLists.body.getCustomizationIdResult.customizationRefList.customizationRef


In [ ]:

response_customList_values = api.service_proxy.getList(
        baseRef=[RecordRef(
            internalId = cl.internalId,
            type="customList",
        )
        for cl in customLists
        ],
        _soapheaders=api.build_soap_headers(),
    )
customList_values = response_customList_values.body.readResponseList.readResponse

### 1a. We need to modify zeep code due to an error in zeep when downloading custom fields

In [ ]:
def stolen_zeep_code_send(self, client, options, operation, args, kwargs):
    envelope, http_headers = self._create(
        operation, args, kwargs, client=client, options=options
    )

    response = client.transport.post_xml(options["address"], envelope, http_headers)

    #operation_obj = self.get(operation)

    # If the client wants to return the raw data then let's do that.
    if client.settings.raw_response:
        return response

    return response

def stolen_zeep_code__call__(api, *args, **kwargs):
    # look at the service proxy class to understand this madness
    self = api.service_proxy._operations["get"]

    soap_headers = self._merge_soap_headers(kwargs.get("_soapheaders"))
    if soap_headers:
        kwargs["_soapheaders"] = soap_headers

    return stolen_zeep_code_send(
        self._proxy._binding,
        self._proxy._client,
        self._proxy._binding_options,
        self._op_name,
        args,
        kwargs,
    )

In [ ]:
response_itemCustomField = api.service_proxy.getCustomizationId(
    customizationType="itemCustomField",
    includeInactives=False,
    _soapheaders = api.build_soap_headers(),
)
itemCustomFields = response_itemCustomField.body.getCustomizationIdResult.customizationRefList.customizationRef


In [ ]:
response = api.service_proxy.search(
searchRecord = ItemSearchBasic(
            type={"operator": "anyOf", "searchValue": ["inventoryItem"]},
    externalId = SearchMultiSelectField(
        operator="anyOf",
        searchValue = [RecordRef(externalId="MEGA-PRODUCT-707374")]
    )),
 _soapheaders=api.build_soap_headers()
)
one_item = response.body.searchResult.recordList.record[0]

In [ ]:
len(one_item.customFieldList.customField) # useless, not enough! cry!

### 1a. get details of each custom field to link it to the custom list (this can be faster with getList

In [ ]:
web_attributes = """custitem_ob_brand_blurbs
custitem_ob_breadcrumb_category
custitem_ob_categories
custitem_ob_category_listing_label
custitem20
custitem_ob_description
custitem_ob_dimensions_content
custitem_ob_directions
custitem_extra_features_content
custitem_ob_fit_and_features_content
custitem_ob_flags
custitem_ob_ingredients_content
custitem_ob_listing_page_name
custitem_material_instructions_content
custitem_ob_model_specsa_flags
custitem_ob_product_listing_label
custitem_ob_product_page_template
custitem_ob_use_and_care_information
custitem_ob_weight_content
custitem_ob_visibilty
custitem_ob_platform_plus
custitem_ob_claire_test
custitem_ob_fashion_width
custitem_ob_fashion_finish
custitem_ob_fashion_set
custitem_ob_fashion_shortest_length
custitem_ob_fashion_longest_length
custitem_ob_clothing_fastening
custitem_ob_sleeve_length
custitem_ob_neckline
custitem_ob_clothing_fit
custitem_ob_trouser_waist
custitem_ob_waistband
custitem_ob_trouser_style
custitem_ob_dimensions
custitem_ob_trim_composition
custitem_hit
custitem_ob_occasion
custitem_ob_care_instructions
custitem_ob_material_composition
custiteml_ob_lining_composition
custitem_ob_accessories_fastening
custitem_cuffs
custitem_ob_heel_type
custitem_ob_sunglasses_flag
custitem_ob_sunglasses_frame
custitem_ob_arm_length
custitem_ob_bridge
custitem_ob_filter_category
custitem_ob_lense_colour
custitem_ob_touch_screen_gloves
custitem_ob_elasticated
custitem_ob_one_size
custitem_ob_card_slots
custitem_closure
custitem_ob_compartment_details
custitem_hardware
custitem_ob_inner_slip_pockets
custitem_ob_lining
custitem_ob_main_compartments
custitem_ob_outer_pockets
custitem_strap_drop
custitem_ob_strap_length
custitem_ob_strap_type
custitem_ob_bag_strap_width
custitem_ob_number_of_straps
custitem_ob_bag_fits
custitem_ob_detachable_strap
custitem_length_extendable
custitem_ob_necklace_length
custitem_ob_chain_type
custitem_ob_stone_length
custitem_ob_stone_width
custitem_ob_drop
custitem_ob_stone
custitem_ob_number_of_chains
custitem_ob_earring_backs
custitem_ob_jewellery_packaging
custitem_ob_jewellery_care_ins
custitem_ob_hair_closure
custitem_ob_headband_profile
custitem_ob_inner_zipped_pockets
custitem_ob_sole_material
custitem_ob_size_guide
custitem_ob_heel_height
custitem_ob_fashion_detail
custitem_ob_fabric_type
custitem_ob_end_use
custitem_ob_clothing_shape
custitem_ob_length
custitem_ob_allow_gift_box
custitem_ob_allow_gift_message
custitem_ob_allow_gift_wrapping
custitem_ob_cross_sell_products
custitem_design
custitem_ob_price_label
custitem_ob_related_products
custitem_ob_sustainability_cta
custitem_ob_sustainability_flags
custitem_ob_upsell_products
custitem_ob_giftbox_product_id
custitem_ob_enable_jewelry_gif
custitem_ob_height
custitem_ob_depth
custitem_ob_width
custitem_ob_weight
custitem_ob_boxed_height_cm
custitem_ob_boxed_width_cm
custitem_ob_boxed_depth_cm
custitem_ob_boxed_weight
custitem_ob_size
custitem_ob_extra_features
custitem_ob_product_collection
custitem_ob_material_breakdown
custitem_ob_pattern
custitem_ob_matching_items_available
custitem_ob_washing_info
custitem_ob_care_info
custitem_ob_safety_info
custitem_ob_lined
custitem_ob_fashion_length
custitem_ob_product_type
custitem_ob_style_attribute
custitem_ob_department_attribute
custitem_ob_brand_attribute
custitem_ob_colour_attributes
custitem_ob_material_attributes
custitem_ob_shower_resistant
custitem_ob_sustainability
custitem_ob_eee
custitem_ob_weee_obligation
custitem_ob_battery
custitem_ob_battery_branded_name
custitem_ob_battery_make_up
custitem_ob_battery_individual_weight
custitem_ob_weee_category
custitem_ob_global_web_finish
custitem_ob_global_web_detail
custitem_ob_hg_detail
custitem_ob_gift_boxed
custitem_ob_hanging_rail
custitem_ob_no_of_hanging_bars
custitem_ob_no_of_earrings_bars
custitem_ob_inside_compartment
custitem_ob_no_of_compartments
custitem_ob_hg_fastening
custitem_ob_shape
custitem_ob_opening
custitem_ob_frame_opening
custitemob_no_of_drawers
custitem_ob_no_of_doors
custitem_ob_no_of_shelves
custitem_ob_adjustable_shelves
custitem_ob_no_of_hooks
custitem_ob_variations
custitem_ob_home_flags
custitem_ob_burn_time
custitem_ob_fill_weight_g
custitem_ob_fill_weight_ml
custitem_ob_ingredients
custitem_ob_allergens
custitemob_directions_of_use
custitem_ob_outdoor_use
custitem_ob_fragrance_name
custitem_ob_fragrance_family_filter
custitem_ob_scent_notes
custitem_ob_hanging_length_cm
custitem_ob_wall_hanging_fixtures
custitem_ob_carry_handles
custitem_ob_bottle_capacity
custitem_ob_glass_capacity
custitem_ob_nesting
custitem_ob_assembly_required
custitem_ob_fluid_capacity_ml
custitem_ob_led_bulb
custitem_ob_recommended_bulb
custitem_ob_bulb_included
custitem_ob_bulb_fitting
custitem_ob_power_source
custitem_ob_conversion_ring_included
custitem_ob_cord_length_cm
custitem_ob_cord_colour
custitem_ob_switch_type
custitem_ob_shade_included
custitem_ob_tint
custitem_ob_maximum_wattage
custitem_ob_hours
custitem_ob_voltage
custitem_ob_kelvin
custitem_ob_lumens
custitem_ob_dimmable
custitem_ob_frame_orientation
custitem_ob_tilt
custitem_ob_magnifying_mirror
custitem_ob_light_up_mirror
custitem_ob_aperture
custitem_ob_plant_pot_height
custitem_ob_plant_pot_width
custitem_ob_plant_pot_depth
custitem_ob_stand_height_cm
custitem_ob_stand_width_cm
custitem_ob_stand_depth_cm
custitem_ob_pot_internal_diameter_cm
custitem_ob_stand_included
custitem_ob_drainage_hole
custitem_ob_plant_included
custitem_ob_faux_bedding
custitem_ob_pot_type
custitem_ob_cushion_inner
custitem_ob_inner_included
custitem_ob_cover_included
custitem_ob_recommended_inner
custitem_ob_print_size
custitem_ob_recessed_frame
custitem_ob_frame_finish
custitem_ob_frame_colour
custitem_ob_mount_colour
custitem_ob_removable_print
custitem_ob_wall_art_design
custitem_ob_wall_art_designer
custitem_ob_ribbon_colour
custitem_ob_hanging_string_colour
custitem_ob_book_subject_filter
custitem_ob_cover_type
custitem_ob_page_count
custitem_ob_author
custitem_ob_publisher
custitem_ob_illustrated
custitem_ob_supplier_description
custitem_ob_quantity_per_pack
custitem_ob_card_occasion_filter
custitem_ob_message_inside
custitem_ob_envelope_colour
custitem_ob_charity_contribution
custitem_ob_card_clasp_type
custitem_ob_recommended_age
custitem_ob_integer
custitem_ob_no_of_players
custitem_ob_diary_type
custitem_ob_cable_connection
custitem_ob_compatibilty
custitem_ob_adjustable
custitem_ob_servings
custitem_ob_suitable_for
custitem_ob_skin_type_suitability
custitem_ob_parabens_sls_free
custitem_ob_leg_height
custitem_ob_floor_to_table_height
custitem_ob_wheels
custitem_ob_foot_adjusters
custitem_ob_seat_height
custitem_ob_seat_depth
custitem_ob_arm_height
custitem_ob_backrest_height
custitem_ob_detachable_feet
custitem_ob_no_of_seats
custitem_ob_upholstered
custitem_ob_back_type
custitem_ob_removable_cushion_covers
custitem_ob_movement_type
custitem_ob_cushion_firmness
custitem_ob_letter
custitem_ob_hg_finish
custitem_ob_hg_length
custitem_ob_hg_set
custitem_ob_hg_width
custitem_ob_hg_longest_length
custitem_ob_hg_shortest_length""".split("\n")

In [ ]:
error_attributes = set("""custitem_ob_product_collection
custitem_ob_skin_type_suitability
custitem_ob_size
custitem_ob_product_collection
custitem_ob_material_breakdown
custitem_ob_sustainability
custitem_ob_sustainability
custitem_ob_hg_detail
custitem_ob_hg_detail
custitem_ob_print_size
custitem_ob_print_size
custitem_ob_skin_type_suitability
custitem_ob_claire_test
custitem_ob_letter
custitem_ob_hg_finish
custitem_ob_letter
custitem_ob_hg_finish
custitem_ob_hg_set
custitem_ob_hg_width
custitem_ob_breadcrumb_category
custitem_ob_categories
custitem_ob_hg_set
custitem_ob_hg_width
custitem_ob_breadcrumb_category
custitem_ob_material_breakdown
custitem_ob_categories
custitem_ob_size
custitem_ob_lining
custitem_ob_fashion_width
custitem_ob_fashion_finish
custitem_ob_claire_test
custitem_ob_fashion_width
custitem_ob_fashion_finish
custitem_ob_trim_composition
custitem_ob_trim_composition
custitem_ob_material_composition
custiteml_ob_lining_composition
custitem_ob_material_composition
custiteml_ob_lining_composition
custitem_ob_lining
custitem_ob_light_up_mirror
custitem_ob_stone
custitem_ob_stone
custitem_ob_fashion_detail
custitem_ob_fashion_detail
custitem_design
custitem_design
custitem_ob_light_up_mirror""".split("\n"))

In [ ]:
good_attributes = set(attr for attr in web_attributes if attr not in error_attributes)

In [ ]:
for e in error_attributes:
    print(e)

In [ ]:
#Unexpected element '{urn:customization_2021_2.setup.webservices.netsuite.com}fieldType
# there is a problem with the api! 
itemCustomFields_with_details = []
c = 0
for itemCustomField in itemCustomFields:
    if itemCustomField.scriptId not in good_attributes:
        print("skip:",itemCustomField.scriptId)
        continue
    c += 1
    print(f"{c}/{len(itemCustomFields)}",itemCustomField.name)
    itemCustomField["details"] = stolen_zeep_code__call__(
        api,
        RecordRef(
            internalId = itemCustomField.internalId,
            type="itemCustomField"
        ),
        _soapheaders=api.build_soap_headers(),
    )
    itemCustomFields_with_details.append(itemCustomField)

#### 1b. add them to product_mapping, and write to file for debugging purposes

In [ ]:
class netsuite_mappings:
    list_info = {}
    list_values = {}
    custom_fields = {}

In [ ]:
from pprint import pprint
for row in itemCustomFields_with_details:
    print(row["internalId"], row.name)
    #pprint(row.details.content.decode("utf-8"))
    xml = ET.fromstring(row.details.content.decode("utf-8"))
    select_record_type = xml.find(".//{*}selectRecordType")

    if select_record_type:
        internalId = select_record_type.get("internalId")
    else:
        internalId = None

    row["listId"] = internalId
    row["fieldType"] = xml.find(".//{*}fieldType").text
    netsuite_mappings.custom_fields[row["internalId"]] = row
    

In [ ]:
netsuite_mappings.custom_fields

In [ ]:
for row in customLists:
    netsuite_mappings.list_info[row.internalId] = row

for row in customList_values:
    if row.record.customValueList is not None:
        netsuite_mappings.list_values[row.record.internalId] = row.record.customValueList.customValue
    else:
        print("Fail:",row.record.name)

### 1c. dataframes for debugging

In [ ]:
from oliverbonas_source_dags.apis.api_netsuite import zeep_object_to_dict
def complete_zeep_df(records):
    record_list = zeep_object_to_dict(list(records))
    return pandas.DataFrame(record_list)
def zeep_df(records):
    record_list = list(records)
    attributes = dir(record_list[0])
    return pandas.DataFrame(
        [
            {
                attr: getattr(r, attr)
                for attr in attributes
            }
            for r in record_list]
    )

In [ ]:
dataframe_per_list = []
for customList_id, values in netsuite_mappings.list_values.items():
    frame = zeep_df(values)
    frame["list_id"] = customList_id
    dataframe_per_list.append(frame)

values_df = pandas.concat(dataframe_per_list)
values_df

In [ ]:
list_df = zeep_df(netsuite_mappings.list_info.values())
list_df

In [ ]:
field_df = zeep_df(netsuite_mappings.custom_fields.values())
field_df

# 3. Download values we need (which are not customLists)

In [ ]:
response = api.service_proxy.search(
        searchRecord=AccountSearchBasic(),
        _soapheaders=api.build_soap_headers()
)
accounts = response.body.searchResult.recordList.record

In [ ]:
complete_zeep_df(accounts)

In [ ]:
response = api.service_proxy.search(
        searchRecord=CostCategorySearchBasic(),
        _soapheaders=api.build_soap_headers()
)
response.body.searchResult.recordList.record

In [ ]:
response = api.service_proxy.search(
        searchRecord=VendorSearchBasic(),
        _soapheaders=api.build_soap_headers()
)
response.body.searchResult.recordList.record

In [ ]:
vendors = response.body.searchResult.recordList.record

vendor_map = {}

for vendor in vendors:
    vendor_map[vendor.entityId] = vendor.internalId
    vendor_map[vendor.legalName] = vendor.internalId
    vendor_map[vendor.companyName] = vendor.internalId

vendor_map

In [ ]:
response = api.service_proxy.search(
        searchRecord=SubsidiarySearchBasic(),
        _soapheaders=api.build_soap_headers()
)

subsidiary_mapping = {
    s.name: s.internalId
    for s in response.body.searchResult.recordList.record
}
subsidiary_mapping

In [ ]:

# https://stackoverflow.com/questions/29771560/how-to-look-up-tax-schedules-using-suitetalk
tax_map = {
    "Cake": 5,
    "Shipping": 4,
    "Reduced": 3,
    "Taxable": 1

}

# 4. Use Code to write Code

In [ ]:
netsuite_mappings.custom_fields.values()


In [ ]:
len(netsuite_mappings.custom_fields.values())

In [ ]:
file = open("use_code_to_write_code.txt", "w")
file.write("")
file.close()
file = open("use_code_to_write_code.txt", "a")
for field in netsuite_mappings.custom_fields.values():
    name = field["name"]
    listId = field["listId"]
    
    
    if listId not in netsuite_mappings.list_info:
        listname = None
    else:
        list_info = netsuite_mappings.list_info[listId]
        listname = list_info["name"]
    
    if listId not in netsuite_mappings.list_values:
        listvalues = []
    else:
        list_values = netsuite_mappings.list_values[listId]
        listvalues = [v["value"] for v in list_values]
    
    FIDY = 50

    if field["fieldType"] in ("_multipleSelect", "_listRecord"):
        func = "M" if field["fieldType"] == "_multipleSelect" else "L"
        #print(field["internalId"], name, "->", listname, f"")
        m = f"""{func}({field['internalId']}, "{field['scriptId']}", """

        firstspaces = " " * (FIDY - len(m))
        secondspaces = " " * (30 - len(name))
        fivevalues = ','.join(listvalues[:5])
        file.write(f"""{m}{firstspaces} # {name} {secondspaces} {listId} {listname} ({fivevalues})
           r),\n""")
    
    elif field["fieldType"] in ("_integerNumber", "_decimalNumber", "_percent"):
        func = "I" if field["fieldType"] == "_integerNumber" else "D"
        x = f"""{func}({field['internalId']}, "{field['scriptId']}", """
        firstspaces = " " * (FIDY - len(x))
        file.write(f"""{x}{firstspaces} # {name}
        r),\n""")
    
    elif field["fieldType"] == "_checkBox":
        x = f"""BooleanCustomFieldRef({field['internalId']}, "{field['scriptId']}", """
        firstspaces = " " * (FIDY - len(x))
        file.write(f"""{x}{firstspaces} # {name}
        True),\n""")
    
    else:
        x = f"""S({field['internalId']}, "{field['scriptId']}", """
        firstspaces = " " * (FIDY - len(x))
        file.write(f"""{x}{firstspaces} # {name}
        r),\n""")
    


### 4b. r code :)

In [ ]:
print("class r:")
everything.columns = everything.columns.str.lower()
for col in everything.columns:
    print("   ",col + " = None")

# 5. data cleaning

In [ ]:
everything.department = everything.department.str.strip().str.title()
everything.new_pgr = everything.new_pgr.str.strip().str.title()
everything.new_sgr = everything.new_sgr.str.strip().str.title()
everything.new_type = everything.new_type.str.strip().str.title()
everything.new_type_detail = everything.new_type_detail.str.strip().str.title()

# 6. functions for custom fields

In [ ]:
fashion

In [ ]:
import random

FILL_EMPTIES = False
MEGA_PRODUCT = False

def is_empty(any_value):
     return (MEGA_PRODUCT
        or type(any_value) == pandas.Series
        or any_value is None
        or (type(any_value) != list and pandas.isna(any_value))
        or any_value == []
        or (type(any_value) == list and is_empty(any_value[0]))
    )

def value_mapping_and_fill_empties(andrei_value, value_if_empty, value_mappings = {}):
    if is_empty(andrei_value):
        if MEGA_PRODUCT or FILL_EMPTIES:
            return value_if_empty
        else:
            return None
    else:
        if andrei_value in value_mappings.keys():
            tom_value = value_mappings[andrei_value]
        else:
            tom_value = andrei_value
        return tom_value

def find_custom_list_and_match_values(custom_field_internal_id, andrei_values: list = [], value_mappings: dict = {}):
    print("################################")
    internalId = str(custom_field_internal_id)
    custom_field = netsuite_mappings.custom_fields[internalId]
    listId = custom_field["listId"]

    if listId not in netsuite_mappings.list_info.keys():
        print("No list found")
        print(netsuite_mappings.custom_fields[internalId])
        raise ValueError("No list found")
        return None
    
    if listId not in netsuite_mappings.list_values.keys():
        print("No list values found")
        print(netsuite_mappings.custom_fields[internalId])
        raise ValueError("No list found")
        return None
    
    custom_list_values = netsuite_mappings.list_values[listId]
    
    netsuite_value_ids = []

    completely_empty = is_empty(andrei_values)
    
    if not completely_empty:
        for andrei_value in andrei_values:
            match = False
            tom_value = value_mapping_and_fill_empties(andrei_value, "", value_mappings)
            for netsuite_value_details in custom_list_values:
                #ic(andrei_value)
                #ic(tom_value)
                if str(tom_value).strip().lower() == netsuite_value_details["value"].strip().lower():
                    netsuite_value_id = netsuite_value_details["valueId"]
                    netsuite_value_ids.append(netsuite_value_id)
                    match = True
            
            if not match:
                print("///////////////////////////")
                print(custom_field.scriptId)
                print(andrei_value)
                print(tom_value)
                print([value_details["value"] for value_details in custom_list_values])
                raise ValueError("A ValueError has never been more appropriate")
    
    if MEGA_PRODUCT or (andrei_values == [] and FILL_EMPTIES):
        # set to everything
        netsuite_value_ids = [v["valueId"] for v in custom_list_values]
    
    return [ListOrRecordRef(internalId= Id) for Id in netsuite_value_ids]
    

def S(internalId, scriptId, value, value_mappings = {}):
    value = value_mapping_and_fill_empties(value, "scaryBONES", value_mappings)
    if value is None: return None

    return StringCustomFieldRef(internalId=internalId, scriptId=scriptId, value=value)

def I(internalId, scriptId, value, value_mappings = {}):
    value = value_mapping_and_fill_empties(value, 999, value_mappings)
    if value is None: return None

    return LongCustomFieldRef(internalId=internalId, scriptId=scriptId, value=value)

def D(internalId, scriptId, value, value_mappings = {}):
    value = value_mapping_and_fill_empties(value, 3.1415926535890932384626433, value_mappings)
    if value is None: return None

    return DoubleCustomFieldRef(internalId=internalId, scriptId=scriptId, value=value)

def L(internalId, scriptId, value_name, value_mappings = {}):
    # below is always a list of either 1 or 0 items
    netsuite_value_id = find_custom_list_and_match_values(internalId, [value_name], value_mappings = value_mappings)
    if netsuite_value_id == []: return None

    return SelectCustomFieldRef(internalId = internalId, scriptId= scriptId, value = random.choice(netsuite_value_id))

def M(internalId, scriptId, value_names, value_mappings = {}):
    value_names = value_names if type(value_names) == list else [value_names]
    netsuite_value_ids = find_custom_list_and_match_values(internalId, value_names, value_mappings = value_mappings)
    if netsuite_value_ids == []: return None

    return SelectCustomFieldRef(internalId = internalId, scriptId= scriptId, value = netsuite_value_ids)

def safe_lower(string_or_none):
    if string_or_none is None:
        return None
    else:
        return string_or_none.lower()
    

# 7. Finally we are ready to upload

In [ ]:
everything

In [ ]:
r = everything.iloc[0]
r


In [ ]:
count_non_null = everything.notnull().sum(axis=1)

# Select the row with the maximum non-null values
everything_sorted =  everything.iloc[count_non_null.argsort()[::-1]]

r = everything_sorted.iloc[1]
r

### you have to put the class r below to get the completions

In [ ]:
class r:
    refnr = None
    pgr_no = None
    sgr_no = None
    type_no = None
    group_no = None
    item_name_standard = None
    item_name_with_variant = None
    department = None
    sub_department = None
    config = None
    new_pgr = None
    new_sgr = None
    new_type = None
    new_type_detail = None
    unit_text = None
    variant_text = None
    item_hierarchy = None
    add_check = None
    analytics_children_no = None
    is_single_item = None
    alphabet_matrix = None
    alphabet_id = None
    bedroom_textiles_matrix = None
    bedroom_textiles_id = None
    clothing_matrix = None
    clothing_id = None
    colour_matrix = None
    colour_id = None
    flavour_matrix = None
    flavour_id = None
    footwear_matrix = None
    footwear_id = None
    frames_matrix = None
    frames_id = None
    paper_matrix = None
    paper_id = None
    ring_size_matrix = None
    ring_size_id = None
    scent_matrix = None
    scent_id = None
    stone_matrix = None
    stone_id = None
    uat_matrix = None
    uat_id = None
    xs_xxl_matrix = None
    xs_xxl_id = None
    xs_s_xl_xxl_matrix = None
    xs_s_xl_xxl_id = None
    department_magento = None
    nightwear_length = None
    lined = None
    nightwear_lining = None
    accessories_fastening = None
    clothing_fastening = None
    fastening_ = None
    hit = None
    shower_resistant = None
    lining_material = None
    care_instructions = None
    pattern = None
    pattern_ = None
    finish = None
    fashion_finish = None
    detail = None
    fashion_detail = None
    matching_items = None
    fashion_matching_items = None
    cuffs = None
    heel_type = None
    sunglasses_frame = None
    filter_category = None
    lense_colour = None
    touch_screen_gloves = None
    set_hg = None
    set_fashion = None
    elasticated = None
    one_size = None
    card_slots = None
    compartment_details = None
    hardware = None
    inner_slip_pockets = None
    lining = None
    main_compartments = None
    outer_pockets = None
    strap_type = None
    number_of_straps = None
    length_extendable = None
    necklace_length = None
    chain_type = None
    hair_closure = None
    inside_compartment = None
    shape = None
    opening = None
    home_variations = None
    wall_hanging_fixtures = None
    carry_handles = None
    bottle_capacity = None
    nesting = None
    power_source = None
    conversion_ring_included = None
    bulb_included = None
    cord_colour = None
    switch_type = None
    shade_included = None
    lighting_tint = None
    maximum_wattage = None
    lighting_hours = None
    voltage = None
    kelvin = None
    dimmable = None
    bulb_fitting = None
    compatible_bulb = None
    frame_aperture = None
    frame_opening = None
    stand_included = None
    drainage_hole = None
    plant_included = None
    faux_bedding = None
    pot_type = None
    cushion_inner = None
    cushion_inner_included = None
    cushion_cover_included = None
    print_size = None
    recessed_frame = None
    frame_finish = None
    frame_colour = None
    mount_colour = None
    removable_print = None
    design = None
    ribbon_colour = None
    string_colour = None
    illustrated = None
    envelope_colour = None
    number_of_players = None
    perishable_product = None
    suitable_for = None
    wheels = None
    foot_adjusters = None
    number_of_seats = None
    back_type = None
    movement_type = None
    cushion_firmness = None
    size_actual_size = None
    outdoor_use = None
    lumens = None
    frame_orientation = None
    wall_art_design = None
    wall_art_designer = None
    cover_type = None
    skin_type_suitability = None
    parabens_and_sls_free = None
    size_filter = None
    brand_filter = None
    inner_zipped_pockets = None
    jewellery_care_instruction = None
    gift_boxed = None
    display_in_catalog_if_oos = None
    display_in_search_if_oos = None
    pre_order_delivery_time = None
    price_label = None
    product_page_template = None
    size_guide = None
    staff_only_product = None
    allow_giftbox = None
    visibility = None
    hazmat = None
    hanging_string = None
    letter = None
    category_listing_label = None
    delivery_flag = None
    colorcustom = None
    jewellery_giftbox_enabled = None
    staff_discount_level = None
    scent = None
    dimensions_clothing = None
    length_56 = None
    length_fewer_skus = None
    height = None
    height_ = None
    home_height = None
    width = None
    width_ = None
    home_width = None
    weight_ = None
    weight_kg = None
    material_composition = None
    mi_detail = None
    clothing_lining = None
    sole_material = None
    heel_height = None
    bridge = None
    strap_drop = None
    strap_length = None
    jewellery_material = None
    shortest_length_fashion = None
    shortest_length_home = None
    longest_length_fashion = None
    longest_length_home = None
    stone_length = None
    stone_width = None
    drop = None
    stone = None
    depth = None
    depth_ = None
    home_depth = None
    boxed_height = None
    boxed_width = None
    boxed_depth = None
    boxed_weight = None
    drawers = None
    doors = None
    compartments = None
    shelves = None
    home_material_breakdown = None
    scent_notes = None
    burn_time = None
    fill_weight_g = None
    fill_weight_ml = None
    hanging_length = None
    cord_length = None
    author = None
    publisher = None
    quantity_per_pack = None
    message_inside = None
    no_of_pieces = None
    servings = None
    leg_height = None
    floor_to_table_height = None
    seat_height = None
    seat_depth = None
    backrest_height = None
    product_features = None
    wash_info_1 = None
    wash_info_2 = None
    wash_info_3 = None
    care_info_1 = None
    care_info_2 = None
    care_info_3 = None
    fragrance_name = None
    fluid_capacity_ml = None
    pot_internal_diameter_cm = None
    page_count = None
    supplier_description = None
    bag_strap_width = None
    extra_features = None
    listing_page_name = None
    allow_gift_message = None
    allow_gift_wrapping = None
    use_and_care_information = None
    ean = None
    safety_info = None
    shipping_group = None
    stat_period = None
    display_name = None
    gift_box_product_id = None
    discontinued = None
    arm_length = None
    ingredients = None
    ingredients_new = None
    allergens = None
    directions_of_use = None
    style = None
    sleeve_length = None
    neckline = None
    occasion = None
    closure = None
    number_of_chains = None
    book_subject = None
    card_occasion = None
    sunglasses_flag = None
    product_collection = None
    home_flag = None
    fragrance_family = None
    colour_filter = None
    material = None
    product_type = None
    dimensions_content = None
    bag_fits = None
    earring_backs = None
    description = None
    fit_and_features_content = None
    directions_content = None
    flags = None
    material_care_instructions = None
    model_spec_flags = None
    sustainability_cta = None
    sustainability_flag = None
    brand_blurb = None
    price = None
    special_price = None
    cost = None
    tier_prices = None
    extra_features_content = None
    jewellery_giftbox_size = None
    jewellery_material_flag = None
    assembly_required = None
    care_flag_1 = None
    care_instruction_flag = None
    config_futura = None
    countryoforigin = None
    countryoforiginfull = None
    supplier = None
    supplier_finance = None
    supplier_no = None
    supplier_item_no = None
    name = None
    name_receipt = None
    pivot_rcode = None
    uswebsite = None
    irelandwebsite = None
    web = None
    hazmatcode = None
    seasonalitycoding = None
    collection = None
    ownbrandbranded = None
    agecheck = None
    pgr = None
    sgr = None
    atv_type = None
    grpno = None
    clothingfit = None
    trouserwaist = None
    trouserstyle = None
    trimcomposition = None
    detachablestrap = None
    numberofplayers = None
    assemblyrequired = None
    atv_ref_number = None
    noofhangingbars = None
    noofearringbars = None
    noofcompartments = None
    noofhooks = None
    ledbulb = None
    tilt = None
    plantpotheightcm = None
    plantpotdepthcm = None
    plantpotwidthcm = None
    standwidthcm = None
    standdepthcm = None
    standheightcm = None
    recommendedage = None
    adjustable = None
    detachablefeet = None
    upholstered = None
    removablecushioncovers = None
    hangingrails = None
    suitablefor = None
    width_home_and_gift = None
    width_fashion = None
    fashion_length = None
    clothing_shape = None
    diary_type = None
    dimensions = None
    nightwear_dimension_top = None
    weight_fashion_g = None
    one_size_futura = None
    allergens_all = None
    kelvin_futura = None
    lumens_futura = None
    no_of_drawers_futura = None
    no_of_pieces_futura = None
    plant_included_futura = None
    recommended_age = None
    removable_cushion_covers = None
    weight_home_and_gift_kg = None
    allergens_home_and_gift = None
    season = None
    season_launch = None
    current_gbp_price = None
    original_gbp_price = None
    current_eur_price = None
    original_eur_price = None
    current_usd_price = None
    original_usd_price = None
    inner = None
    outer = None
    cbm = None
    asset_account_constant = None
    income_account_constant = None
    cogs_account_constant = None
    cost_price_constant = None
    cost_category_constant = None
    cost_estimate_type_constant = None
    subsidiary_constant = None
    include_children_constant = None
    tax_schedule_req_constant = None
    matrix_type_constant = None
    track_landed_cost_constant = None
    season_launch_constant = None
    item__hierarchy_versions_1_hierarchy_version_constant = None
    item__hierarchy_versions_1_included_in_version_constant = None
    pricing_1_currency_constant = None
    pricing_1_pricing_level_constant = None
    pricing_1_quantity_constant = None
    pricing_2_currency_constant = None
    pricing_2_pricing_level_constant = None
    pricing_2_quantity_constant = None
    pricing_3_currency_constant = None
    pricing_3_pricing_level_constant = None
    pricing_3_quantity_constant = None
    bag_strap_width_netsuite = None
    bridge_netsuite = None
    ean_netsuite = None
    arm_length_netsuite = None
    ownbrandbranded_netsuite = None
    care_instructions_netsuite = None
    dimensions_netsuite = None
    agecheck_netsuite = None
    drop_netsuite = None
    weight_netsuite = None
    weight_unit_netsuite = None
    longest_length_fashion_netsuite = None
    shortest_length_fashion_netsuite = None
    heel_height_netsuite = None
    one_size_netsuite = None
    size_guide_netsuite = None
    sole_material_netsuite = None
    stone_length_netsuite = None
    stone_width_netsuite = None
    strap_drop_netsuite = None
    strap_length_netsuite = None
    allergens_netsuite = None



In [ ]:
from multithread_uploader import upload_all_these_records

    
"""webattributes = [
        M(4243, "custitem_ob_sustainability_cta",          # Sustainability CTA			           1671 Sustainability CTA			 (BC Cotton CTA,BCI Cotton CTA,EcoVero CTA)
           r.sustainability_cta),
    M(4244, "custitem_ob_sustainability_flags",        # Sustainability Flags			         1672 Sustainability Flags			 (BCI Cotton,EcoVero,Global Organic Textile Standard (GOTS) ,GRS & RCS,EcoVero CTA)
           r.sustainability_flag),
    S(3988, "custitem_ob_boxed_height_cm",             # Boxed height cm
            r.boxed_height),
    S(3989, "custitem_ob_boxed_width_cm",              # Boxed width cm
            r.boxed_width),
    S(3990, "custitem_ob_boxed_depth_cm",              # Boxed depth cm
            r.boxed_depth),
    I(3991, "custitem_ob_boxed_weight",                # Boxed weight G/KG
            r.boxed_weight),
    L(4140, "custitem_ob_brand_attribute",             # Brand Attribute                 1274 Brand Attribute (Beauty Pro,Bloom & Blossom,Bordallo Pinheiro,Coco & Eve,Cocoba)
               r.Brand_filter),
    M(4002, "custitem_ob_care_info",                   # Care Info                       1147 Care info (Avoid contact with eyes,Do not exceed maximum wattage,Do not leave in standing water,Do not use abrasive cleaners,Do not use furniture polish)
               r.care_info_1),# todo: what are other care infos?
    M(4145, "custitem_ob_colour_attributes",           # Colour Attributes               1481 Colour Attributes (Black,Blue,Brown,Clear,Copper)
               r.colour_matrix),
    L(4139, "custitem_ob_department_attribute",        # Department Attribute            1273 Department Attribute (Home,Beauty,Gift,Furniture,Fashion)
               r.department_magento),
    I(4066, "custitem_ob_stand_height_cm",             # Stand height cm
            r.height),
    I(4067, "custitem_ob_stand_width_cm",              # Stand width cm
            r.width),
    I(4068, "custitem_ob_stand_depth_cm",              # Stand depth cm
            r.depth),
    S(3993, "custitem_ob_extra_features",              # Extra Features
            r.extra_features),
    ]"""

vals = r[['department', 'new_pgr', 'new_sgr', 'new_type', 'new_type_detail']]
parent_chain = vals[~pandas.isna(vals)].tolist()
merch_hierarchy_node = " : ".join(parent_chain)

webattributes = [L(4060, "custitem_ob_magnifying_mirror",           # Magnifying mirror               1208 Magnifying mirror (Yes)
           r), # todo null
L(4062, "custitem_ob_aperture",                    # Aperture                        1210 Aperture (Double,Multi,Single)
           r.frame_aperture),
D(4063, "custitem_ob_plant_pot_height",            # Plant pot height
        r.plantpotheightcm),
D(4064, "custitem_ob_plant_pot_width",             # Plant pot width
        r.plantpotwidthcm),
D(4065, "custitem_ob_plant_pot_depth",             # Plant pot depth
        r.plantpotdepthcm),
I(4066, "custitem_ob_stand_height_cm",             # Stand height cm
        r.standheightcm),
I(4067, "custitem_ob_stand_width_cm",              # Stand width cm
        r.standwidthcm),
I(4068, "custitem_ob_stand_depth_cm",              # Stand depth cm
        r.standdepthcm),
D(4069, "custitem_ob_pot_internal_diameter_cm",    # Pot internal diameter cm
        r.pot_internal_diameter_cm),
L(4070, "custitem_ob_stand_included",              # Stand included                  1218 Stand included (Yes,No)
           r.stand_included),
L(4071, "custitem_ob_drainage_hole",               # Drainage hole                   1219 Drainage hole (Yes,No)
           r.drainage_hole),
L(4072, "custitem_ob_plant_included",              # Plant included                  1220 Plant included (No,Yes)
           r.plant_included),
L(4073, "custitem_ob_faux_bedding",                # Faux bedding                    1221 Faux bedding (Moss,Pebbles,Soil)
           r.faux_bedding),
L(4074, "custitem_ob_pot_type",                    # Pot type                        1222 Pot type (Plastic,Clay)
           r.pot_type),
L(4075, "custitem_ob_cushion_inner",               # Cushion inner                   1223 Cushion inner (Polyester)
           r.cushion_inner),
L(4076, "custitem_ob_inner_included",              # Inner included                  1224 Inner included (Yes,No)
           r.cushion_inner_included),
L(4077, "custitem_ob_cover_included",              # Cover included                  1225 Cover included (No)
           r.cushion_cover_included),
L(4078, "custitem_ob_recommended_inner",           # Recommended inner               1226 Recommended inner (30x50,32x95,45x45)
           r), # todo: null
L(4080, "custitem_ob_recessed_frame",              # Recessed frame                  1228 Recessed frame (Yes)
           r.recessed_frame),
L(4081, "custitem_ob_frame_finish",                # Frame finish                    1229 Frame finish (Satin,Matte)
           r.frame_finish),
L(4082, "custitem_ob_frame_colour",                # Frame colour                    1230 Frame colour (Black,Gold,Natural,White)
           r.frame_colour),
L(4083, "custitem_ob_mount_colour",                # Mount colour                    1231 Mount colour (Black,Cream,White)
           r.mount_colour),
L(4084, "custitem_ob_removable_print",             # Removable print                 1232 Removable print (Yes,No)
           r.removable_print),
L(4085, "custitem_ob_wall_art_design",             # Wall art design                 1233 Wall art design (Abstract,Animal,Graphic,Landscape,London)
           r.wall_art_design),
L(4086, "custitem_ob_wall_art_designer",           # Wall art designer               1234 Wall art designer (All the ways to say,Fox & Velvet,Henry Rivers,HollieGraphik,Honeymoon Hotel)
           r.wall_art_designer),
L(4087, "custitem_ob_ribbon_colour",               # Ribbon colour                   1235 Ribbon colour (Black,Blue,Brown,Clear,Copper)
           r.ribbon_colour),
L(4088, "custitem_ob_hanging_string_colour",       # Hanging string colour           1376 Hanging string colour (Black,Blue,Brown,Clear,Copper)
           r.string_colour),
M(4089, "custitem_ob_book_subject_filter",         # Book subject filter             1237 Book subject filter (Books For Change,Children's Books,Colouring Books,Design & Art Books,Feminist Books)
           r.book_subject),
L(4090, "custitem_ob_cover_type",                  # Cover type                      1238 Cover type (Hardback,Paperback)
           r.cover_type),
S(4091, "custitem_ob_page_count",                  # Page count
        r.page_count),
S(4092, "custitem_ob_author",                      # Author
        r.author),
S(4093, "custitem_ob_publisher",                   # Publisher
        r.publisher),
L(4094, "custitem_ob_illustrated",                 # Illustrated                     1241 Illustrated (Yes)
           r.illustrated),
S(4095, "custitem_ob_supplier_description",        # Supplier description
        r.supplier_description),
S(4096, "custitem_ob_quantity_per_pack",           # Quantity per pack
        r.quantity_per_pack),
M(4097, "custitem_ob_card_occasion_filter",        # Card occasion filter            1242 Card occasion filter (Anniversary,Art & Illustration,Baby,Birthday,Christening)
           r.card_occasion),
S(4098, "custitem_ob_message_inside",              # Message inside
        r.message_inside),
L(4099, "custitem_ob_envelope_colour",             # Envelope colour                 1243 Envelope colour (Black,Blue,Brown,Clear,Copper)
           r.envelope_colour),
L(4100, "custitem_ob_charity_contribution",        # Charity contribution            1244 Charity contribution (Yes)
           r), # todo: null
L(4101, "custitem_ob_card_clasp_type",             # Card clasp type                 1245 Card clasp type (Biodegradable cellophane,Cellophane,Nested,Peelable sticker)
           r), # todo: null
L(4102, "custitem_ob_recommended_age",             # Recommended age                 1246 Recommended age (3+,4+,5+,6+,7+)
           r.recommended_age),
S(4103, "custitem_ob_integer",                     # No. of pieces
        r.no_of_pieces),
L(4104, "custitem_ob_no_of_players",               # No. of players                  1247 No. of players (2+ players,2-8 players,3-6 players,4+ players,4-8 players)
           r.number_of_players),
L(4105, "custitem_ob_diary_type",                  # Diary type                      1248 Diary type (Academic,Yearly)
           r.diary_type),
M(4106, "custitem_ob_cable_connection",            # Cable connection                1249 Cable connection (USB)
           r), # todo: null
M(4107, "custitem_ob_compatibilty",                # Compatibility                   1250 Compatibility (iPhone 11,iPhone 11 Pro,iPhone 11 Pro Max,iPhone 12,iPhone 12 Mini)
           r), # todo: null
L(4108, "custitem_ob_adjustable",                  # Adjustable                      1251 Adjustable (Yes,No)
           r.adjustable),
S(4109, "custitem_ob_servings",                    # Servings
        r.servings),
M(4111, "custitem_ob_suitable_for",                # Suitable for ?                  1254 Suitable for ? (Gluten free,Vegans,Vegetarians,Coeliacs,Cats)
           r.suitable_for),
L(4118, "custitem_ob_parabens_sls_free",           # Parabens & sls free             1257 Parabens & sls free (Yes)
           r.parabens_and_sls_free),
I(4119, "custitem_ob_leg_height",                  # Leg height
        r.leg_height),
I(4120, "custitem_ob_floor_to_table_height",       # Floor to table height
        r.floor_to_table_height),
L(4121, "custitem_ob_wheels",                      # Wheels                          1258 Wheels (Yes)
           r.wheels),
L(4122, "custitem_ob_foot_adjusters",              # Foot adjusters                  1259 Foot adjusters (Yes)
           r.foot_adjusters),
I(4123, "custitem_ob_seat_height",                 # Seat height
        r.seat_height),
I(4124, "custitem_ob_seat_depth",                  # Seat depth
        r.seat_depth),
I(4125, "custitem_ob_arm_height",                  # Arm height
        r), # todo: null
I(4126, "custitem_ob_backrest_height",             # Backrest height
        r.backrest_height),
L(4128, "custitem_ob_detachable_feet",             # Detachable feet                 1260 Detachable feet (Yes,No)
           r.detachablefeet),
L(4129, "custitem_ob_no_of_seats",                 # No. of seats                    1261 No. of seats (One,One and a half,Two)
           r.number_of_seats),
L(4130, "custitem_ob_upholstered",                 # Upholstered                     1262 Upholstered (Yes,No)
           r.upholstered),
L(4131, "custitem_ob_back_type",                   # Back type                       1263 Back type (Button,Cushion,Fixed,High back,Wingback)
           r.back_type),
L(4132, "custitem_ob_removable_cushion_covers",    # Removable cushion covers        1264 Removable cushion covers (Yes,No)
           r.removable_cushion_covers),
L(4133, "custitem_ob_movement_type",               # Movement type                   1265 Movement type (Fixed,Reclines,Swivel)
           r.movement_type),
L(4134, "custitem_ob_cushion_firmness",            # Cushion firmness                1266 Cushion firmness (Firm,Soft)
           r.cushion_firmness),
S(4265, "custitem_ob_hg_length",                   # H&G Length
        r.length_56), # todo: funny
S(4269, "custitem_ob_hg_longest_length",           # H&G Longest Length
        r.longest_length_home),
S(4270, "custitem_ob_hg_shortest_length",          # H&G Shortest Length
        r.shortest_length_home),
L(4211, "custitem_ob_brand_blurbs",                # Brand Blurbs			                 1651 Brand Blurbs (Aloha,Basket Room,Esska,Gola,HOFF)
           r.brand_blurb),
L(4214, "custitem_ob_category_listing_label",      # Category Listing Label          1654 Category Listing Label (AVAILABLE NOW,MADE TO ORDER,Gold Plated,MIX & MATCH LIGHTING: This style is part of our Mix & Matching Lighting collection. Customise your own lamp with a range of bases and colourful shades to choose from. ,NEW TO SALE)
           r.category_listing_label),
L(4216, "custitem20",                              # Content Department              1656 Content Department (Home,Gift,Beauty,Furniture,Fashion)
           r.sub_department),
S(4217, "custitem_ob_description",                 # Description
        r.description),
S(4219, "custitem_ob_dimensions_content",          # Dimensions (Content)
        r.dimensions_content),
S(4220, "custitem_ob_directions",                  # Directions (Content)
        r.directions_content),
S(4223, "custitem_extra_features_content",         # Extra Features (Content)
        r.extra_features_content),
S(4227, "custitem_ob_fit_and_features_content",    # Fit & Features Content
        r.fit_and_features_content),
M(4229, "custitem_ob_flags",                       # Flags                           1662 Flags			 (40W Energy Class,Acetate Sunglasses,Acetate Sunglasses Buying Guide,Always check the care label before laundering,Buying Guide: Face Masks)
           r.flags),
S(4230, "custitem_ob_ingredients_content",         # Ingredients (Content)
        r.ingredients), # todo: ingreedients new
S(4231, "custitem_ob_listing_page_name",           # Listing Page Name
        r.listing_page_name),
S(4233, "custitem_material_instructions_content",  # Material & Instructions (Content)
        r.material_care_instructions),
M(4234, "custitem_ob_model_specsa_flags",          # Model Specs Flags			            1663 Model Specs Flags			 (Apple,Carter,Felicity,Gabrielle,Gigi)
           r.model_spec_flags),
L(4238, "custitem_ob_product_listing_label",       # Product Listing Label	          1666 Product Listing Label	 (All Fabric Swatches,Bag Strap,Card Delivery Cost,Cushion Inner 30x50cm,Cushion Inner 45x45cm)
           r.category_listing_label),
L(4239, "custitem_ob_product_page_template",       # Product page template           1667 Product page template (Accessories,Accessories_attribute,Cutouts,Fashion_model,Fashion_model_attribute)
           r.product_page_template),
S(4246, "custitem_ob_use_and_care_information",    # Use and Care Information
        r.use_and_care_information),
S(4247, "custitem_ob_weight_content",              # Weight (Content)
        r), # todo: null
L(4248, "custitem_ob_visibilty",                   # Visibility                      1674 Visibility (Catalog, Search,Catalog,Search,Not Visible Individually)
           r.visibility),
BooleanCustomFieldRef(4275, "custitem_ob_platform_plus",  # Live for Platform Plus
        True), # todo: null
L(3999, "custitem_ob_fashion_set",                 # Fashion Set                     1151 Set (Gift set,Set of two,Set of three,Set of four,Set of five)
           r.set_fashion),
S(4113, "custitem_ob_fashion_shortest_length",     # Fashion Shortest length
        r.shortest_length_fashion),
S(4114, "custitem_ob_fashion_longest_length",      # Fashion Longest Length
        r.longest_length_fashion_netsuite), # todo: _netsuite? and _futura
M(4141, "custitem_ob_clothing_fastening",          # Clothing fastening              1477 Clothing fastening (Belt,Buckle,Button,Fly,Loop)
           r.clothing_fastening),
L(4142, "custitem_ob_sleeve_length",               # Sleeve length                   1478 Sleeve length (Long,Puff,Short,Sleeveless,Strappy)
           r.sleeve_length),
L(4143, "custitem_ob_neckline",                    # Neckline                        1479 Neckline (Collar,Crew neck,Funnel neck,Halter neck,High neck)
           r.neckline),
M(4151, "custitem_ob_clothing_fit",                # Clothing fit                    1480 Clothing fit (Ankle Length,Ankle slim,Baggy,Bodycon,Bolero)
           r.clothingfit),
L(4152, "custitem_ob_trouser_waist",               # Trouser waist                   1483 Trouser waist (Low rise,Mid rise,High waist)
           r.trouserwaist),
L(4153, "custitem_ob_waistband",                   # Waistband                       1484 Waistband (Paperbag,Elasticated,Tie)
           r), # todo: null
L(4154, "custitem_ob_trouser_style",               # Trouser style                   1485 Trouser style (Bootcut,Boyfriend,Cigarette,Flared,Girlfriend)
           r.trouserstyle),
S(4155, "custitem_ob_dimensions",                  # Dimensions
        r.dimensions_netsuite),
L(4157, "custitem_hit",                            # Hit                             1496 Hit (1,2,3,4,5)
           r.hit),
M(4159, "custitem_ob_occasion",                    # Occasion                        1498 Occasion (Casual,Day dressing,Evening,Festival,Going out)
           r.occasion),
M(4160, "custitem_ob_care_instructions",           # Care instructions               1500 Care instructions (Always check the care label before laundering,Machine washable,Dry clean only,Do not wash,Wipe clean only)
           r.care_instructions_netsuite),
L(4164, "custitem_ob_accessories_fastening",       # Accessories fastening           1492 Accessories fastening (Belt,Buckle,Button,Fly,Laces)
           r.accessories_fastening),
L(4165, "custitem_cuffs",                          # Cuffs                           1493 Cuffs (Elasticated,Turn up)
           r.cuffs),
L(4166, "custitem_ob_heel_type",                   # Heel type                       1547 Heel type (Blocked heel,Flat,Flatform,Wedge,Skinny heel)
           r.heel_height_netsuite),
M(4167, "custitem_ob_sunglasses_flag",             # Sunglasses flag                 1504 Sunglasses flag (Sunglasses CE Mark,Acetate Sunglasses,Acetate Sunglasses Buying Guide,Sunglasses Pouch,Sunglasses Case)
           r.sunglasses_flag),
L(4168, "custitem_ob_sunglasses_frame",            # Sunglasses frame                1506 Sunglasses frame (Full frame,Half frame,Metal,Rimless)
           r.sunglasses_frame),
D(4169, "custitem_ob_arm_length",                  # Arm length
        r.arm_length_netsuite),
L(4170, "custitem_ob_bridge",                      # Bridge                          1508 Bridge (1 cm)
           r.bridge_netsuite),
L(4171, "custitem_ob_filter_category",             # Filter category                 1509 Filter category (3,0)
           r.filter_category),
L(4172, "custitem_ob_lense_colour",                # Lense colour                    1510 Lens colour (Black,Blue,Brown,Grey,Ombre)
           r.lense_colour),
L(4173, "custitem_ob_touch_screen_gloves",         # Touch screen gloves             1511 Touch screen gloves (Yes,No)
           r.touch_screen_gloves),
L(4174, "custitem_ob_elasticated",                 # Elasticated                     1512 Elasticated (Yes)
           r.elasticated),
L(4175, "custitem_ob_one_size",                    # One size                        1513 One size (Yes)
           r.one_size_netsuite),
L(4176, "custitem_ob_card_slots",                  # Card slots                      1514 Card slots (1,2,3,4,5)
           r.card_slots),
L(4177, "custitem_closure",                        # Closure                         1515 Closure (Buttons,Clasp,Concealed magnetic,Concealed magnetic closure,Continuous)
           r.closure),
M(4178, "custitem_ob_compartment_details",         # Compartment details             1516 Compartment details (ID window,Internal pouch,Mobile pocket,Removable cardholder and ID window,Removable cardholder)
           r.compartment_details),
L(4179, "custitem_hardware",                       # Hardware                        1517 Hardware (Brushed gold toned,Brushed silver toned,Gold,Gold toned,Gunmetal)
           r.hardware),
L(4180, "custitem_ob_inner_slip_pockets",          # Inner slip pockets              1518 Inner slip pockets (1,2,3,4)
           r.inner_slip_pockets),
L(4182, "custitem_ob_main_compartments",           # Main compartments               1521 Main compartments (1,2,3)
           r.main_compartments),
L(4183, "custitem_ob_outer_pockets",               # Outer pockets                   1522 Outer pockets (1,2,3)
           r.outer_pockets),
S(4184, "custitem_strap_drop",                     # Strap drop CM
        r.strap_drop),
D(4185, "custitem_ob_strap_length",                # Strap length
        r.strap_length_netsuite),
L(4186, "custitem_ob_strap_type",                  # Strap type                      1525 Strap type (Adjustable strap,Fixed strap,No strap,Two strap,Wrist strap)
           r.strap_type),
D(4187, "custitem_ob_bag_strap_width",             # Bag strap width 
        r.bag_strap_width),
L(4188, "custitem_ob_number_of_straps",            # Number of straps                1527 Number of straps (1,2,3,4,5)
           r.number_of_straps),
M(4189, "custitem_ob_bag_fits",                    # Bag fits                        1528 Bag fits (13 inch laptop,15 inch laptop,A4 notebook,Small tablet,Large tablet)
           r.bag_fits),
L(4190, "custitem_ob_detachable_strap",            # Detachable strap                1529 Detachable strap (Yes,No)
           r.detachablestrap),
L(4191, "custitem_length_extendable",              # Length extendable               1530 Length extendable (Yes,No)
           r.length_extendable),
L(4192, "custitem_ob_necklace_length",             # Necklace length                 1873 Necklace Length (Collar,Long,Short)
           r.necklace_length),
L(4193, "custitem_ob_chain_type",                  # Chain type                      1532 Chain type (Anchor,Ball,Beaded,Box,Cable)
           r.chain_type),
D(4195, "custitem_ob_stone_length",                # Stone length
        r.stone_length_netsuite),
D(4196, "custitem_ob_stone_width",                 # Stone width
        r.stone_width_netsuite),
S(4197, "custitem_ob_drop",                        # Drop CM
        r.drop_netsuite),
L(4199, "custitem_ob_number_of_chains",            # Number of chains                1538 Number of chains (Double,Single,Triple,Multistrand)
           r.number_of_chains),
L(4200, "custitem_ob_earring_backs",               # Earring backs                   1540 Earring backs (Clicker,Hook,Latch,Stud,Thread through)
           r.earring_backs),
L(4201, "custitem_ob_jewellery_packaging",         # Jewellery packaging             1541 Jewellery packaging (Yes,No)
           r), # todo null
M(4202, "custitem_ob_jewellery_care_ins",          # Jewellery care instructions     1542 Jewellery care instructions (Jewellery Care: Costume,Jewellery Care: Gold Plated,Jewellery Care: Silver Plated,Jewellery Care: Semi Precious,Jewellery Care: Silver 925)
           r.jewellery_care_instruction),
L(4203, "custitem_ob_hair_closure",                # Hair closure                    1543 Hair closure (Barette,Clasp,Claw,Slide,Snap)
           r.hair_closure),
L(4204, "custitem_ob_headband_profile",            # Headband profile                1544 Headband profile (Beaded,Bow,Embellished,Knot top,Statement bow)
           r), # todo null
L(4205, "custitem_ob_inner_zipped_pockets",        # Inner zipped pockets            1519 Inner zipped pockets (1,2)
           r.inner_zipped_pockets),
L(4206, "custitem_ob_sole_material",               # Sole material                   1495 Sole material (EVA,Jute,Leather,Other materials,Raffia)
           r.sole_material_netsuite),
L(4241, "custitem_ob_size_guide",                  # Size Guide                      1669 Size Guide (Belt,Shoe,Tights,Gloves,Ring)
           r.size_guide_netsuite),
D(4262, "custitem_ob_heel_height",                 # Heel height
        r.heel_height_netsuite),
L(4839, "custitem_ob_fabric_type",                 # Fabric Type                     1861 Fabric Type (Print,Plain,Texture)
           r), # todo where is it?
L(4840, "custitem_ob_end_use",                     # End Use                         1863 End Use (Smart,Casual,Smart/Casual)
           r), # todo where is it?
L(4841, "custitem_ob_clothing_shape",              # Clothing Shape                  1862 Clothing Shape (Mini Dresses,Midi Dresses,Maxi Dresses,Shirt Dresses,Shift Dresses)
           r.clothing_shape),
D(4853, "custitem_ob_length",                      # Length
        r), # todo, what length?
BooleanCustomFieldRef(4207, "custitem_ob_allow_gift_box",  # Allow Gift Box
        r.allow_giftbox),
L(4208, "custitem_ob_allow_gift_message",          # Allow Gift Message              1648 Allow Gift Message (Yes,No,Use config)
           r.allow_gift_message),
L(4209, "custitem_ob_allow_gift_wrapping",         # Allow Gift Wrapping             1649 Allow Gift Wrapping (Yes,No,Use config)
           r.allow_gift_wrapping),
L(4215, "custitem_ob_cross_sell_products",         # Cross-Sell Products             1655 Cross-Sell Products (Yes)
           r), # todo null
L(4237, "custitem_ob_price_label",                 # Price Label                     1665 Price Label (Markdown,Full Price)
           r.price_label),
M(4240, "custitem_ob_related_products",            # Related Products                1668 Related Products (1234689)
           r), # what? todo null
M(4243, "custitem_ob_sustainability_cta",          # Sustainability CTA			           1671 Sustainability CTA			 (BC Cotton CTA,BCI Cotton CTA,EcoVero CTA)
           r.sustainability_cta),
M(4244, "custitem_ob_sustainability_flags",        # Sustainability Flags			         1672 Sustainability Flags			 (BCI Cotton,EcoVero,Global Organic Textile Standard (GOTS) ,GRS & RCS,EcoVero CTA)
           r.sustainability_flag),
L(4245, "custitem_ob_upsell_products",             # Up-Sell Products                1673 Up-Sell Products (Yes)
           r), # todo null
S(4271, "custitem_ob_giftbox_product_id",          # Giftbox Product Id
        r.gift_box_product_id),
BooleanCustomFieldRef(4567, "custitem_ob_enable_jewelry_gif",  # Enable Jewellery Giftbox
        r.jewellery_giftbox_enabled),
S(3983, "custitem_ob_height",                      # Height
        r.height),
S(3986, "custitem_ob_depth",                       # Depth
        r.depth),
S(4901, "custitem_ob_width",                       # Width CM
        r.width),
S(3987, "custitem_ob_weight",                      # Weight G/KG
        r.weight_netsuite),
S(3988, "custitem_ob_boxed_height_cm",             # Boxed height cm
        r.boxed_height),
S(3989, "custitem_ob_boxed_width_cm",              # Boxed width cm
        r.boxed_width),
S(3990, "custitem_ob_boxed_depth_cm",              # Boxed depth cm
        r.boxed_depth),
D(3991, "custitem_ob_boxed_weight",                # Boxed weight G/KG
        r.boxed_weight),
S(3993, "custitem_ob_extra_features",              # Extra Features
        r.extra_features), # todo: content?
M(3996, "custitem_ob_pattern",                     # Pattern                         1141 Pattern (Abstract,Animal print,Artist,Character,Check)
           r.pattern),
L(4000, "custitem_ob_matching_items_available",    # Matching Items Available        1145 Matching items available (Yes,No)
           r.matching_items),
M(4001, "custitem_ob_washing_info",                # Washing Info                    1146 Washing info (Air dry only,Always wash separately,Clean with a damp cloth, followed by a soft dry cloth,Cool hand wash only,Dishwasher and microwave safe)
           [r.wash_info_1, r.wash_info_2, r.wash_info_3]), # just 1?
M(4002, "custitem_ob_care_info",                   # Care Info                       1147 Care info (Avoid contact with eyes,Do not exceed maximum wattage,Do not leave in standing water,Do not use abrasive cleaners,Do not use furniture polish)
           [r.care_info_1, r.care_info_2, r.care_info_3]),
M(4003, "custitem_ob_safety_info",                 # Safety Info                     1148 Safety info (Do not move burning candles,Domestic use only,For decorative use only,For indoor use only,For use with dry flowers only)
           r.safety_info),
M(4012, "custitem_ob_lined",                       # Lined                           1160 Lined (Bonded,Contrast lining,Fully lined,No,Partially lined)
           r.lined),
L(4116, "custitem_ob_fashion_length",              # Fashion Length                  1546 Fashion Length (Midi,Knee,Midaxi,Maxi,Mini)
           r.fashion_length),
L(4135, "custitem_ob_product_type",                # Product Type                    1267 Product Type (Activewear,Anklets,Artificial Plants,Bags,Bath)
           r.product_type),
L(4138, "custitem_ob_style_attribute",             # Style Attribute                 1272 Style Attribute (Airpod Cases,Armchairs,Backpacks,Bag Straps,Baker Boy Hats)
           r.style),
L(4139, "custitem_ob_department_attribute",        # Department Attribute            1273 Department Attribute (Home,Beauty,Gift,Furniture,Fashion)
           r.department_magento),
L(4140, "custitem_ob_brand_attribute",             # Brand Attribute                 1274 Brand Attribute (Beauty Pro,Bloom & Blossom,Bordallo Pinheiro,Coco & Eve,Cocoba)
           r.brand_filter), # 
M(4145, "custitem_ob_colour_attributes",           # Colour Attributes               1481 Colour Attributes (Black,Blue,Brown,Clear,Copper)
           r.colour_matrix),
M(4147, "custitem_ob_material_attributes",         # Material Attributes             1482 Material Attributes (Bamboo,Brass,Broderie Anglaise,Card,Cashmere)
           r.material),
L(4158, "custitem_ob_shower_resistant",            # Shower Resistant                1497 Shower resistant (Yes,No)
           r.shower_resistant),
BooleanCustomFieldRef(4831, "custitem_ob_eee",     # EEE
        True), # todo null
BooleanCustomFieldRef(4832, "custitem_ob_weee_obligation",  # WEEE Obligation
        True), # todo null
BooleanCustomFieldRef(4833, "custitem_ob_battery",  # Battery
        True), # todo null
S(4834, "custitem_ob_battery_branded_name",        # Battery Branded Name
        r), # todo null
S(4836, "custitem_ob_battery_make_up",             # Battery Make Up
        r), # todo null
S(4837, "custitem_ob_battery_individual_weight",   # Battery Individual Weight Grams
        r), # todo null
L(4838, "custitem_ob_weee_category",               # WEEE Category                   1860 Weee Category (Appliances Container Refrigerants,Automatic Dispensers,Consumer Equipment,Display Equipment,Electrical and Electronic tools)
           r), # todo null
L(4843, "custitem_ob_global_web_finish",           # Global Web Attributes Finish    1864 Global Web Attributes Finish (Boucle,Braided,Canvas,Crochet,Embossed)
           r.finish), # todo global web?
L(4844, "custitem_ob_global_web_detail",           # Global Web Attributes Detail    1866 Global Web Attributes Detail (Applique,Badges,Beaded,Block printed,Digital printed)
           r.detail), # todo: as above
L(4004, "custitem_ob_gift_boxed",                  # Gift boxed                      1152 Gift Boxed (Brown Box,White Box,No,Gift Box)
           r.gift_boxed),
L(4005, "custitem_ob_hanging_rail",                # Hanging rail                    1153 Hanging rail (Yes)
           r.hangingrails),
L(4006, "custitem_ob_no_of_hanging_bars",          # No. of hanging bars             1154 No. of hanging bars (1,2,3,4,5)
           r.noofhangingbars),
L(4007, "custitem_ob_no_of_earrings_bars",         # No. of earrings bars            1155 No. of earrings bars (1,2,3,4,5)
           r.noofearringbars),
M(4008, "custitem_ob_inside_compartment",          # Inside compartment              1156 Inside compartment (Divided sections,Earring storage,Pockets,Ring compartment)
           r.inside_compartment),
L(4009, "custitem_ob_no_of_compartments",          # No. of compartments             1157 No. of compartments (1,2,3,4,5)
           r.noofcompartments),
L(4010, "custitem_ob_hg_fastening",                # Fastening                       1158 Fastening (Buttons,Clasp,Drawstring,Popper buttons,Ties)
           r.clothing_fastening),
L(4011, "custitem_ob_shape",                       # Shape                           1159 Shape (Animal,Arch,Cone,Heart,Hexagonal)
           r.shape[0], {422: "Animal"}),
L(4013, "custitem_ob_opening",                     # Opening                         1161 Opening (Door,Drawer,Lid)
           r.opening),
L(4014, "custitem_ob_frame_opening",               # Frame opening                   1162 Frame opening (Hinge,Hook,Slide)
           r.frame_opening),
L(4015, "custitemob_no_of_drawers",                # No. of drawers                  1163 No. of drawers (1,2,3,4,5)
           r.drawers),
L(4016, "custitem_ob_no_of_doors",                 # No. of doors                    1164 No. of doors (1,2)
           r.doors),
L(4017, "custitem_ob_no_of_shelves",               # No. of shelves                  1165 No. of shelves (1,2,3,4,5)
           r.shelves),
L(4018, "custitem_ob_adjustable_shelves",          # Adjustable shelves              1166 Adjustable shelves (Yes,No)
           r), # todo null
L(4019, "custitem_ob_no_of_hooks",                 # No. of hooks                    1167 No. of hooks (1,2,3,4,5)
           r.noofhooks),
L(4020, "custitem_ob_variations",                  # Variations                      1168 Variations (Variations may occur)
           r.home_variations),
M(4021, "custitem_ob_home_flags",                  # Home flags                      1169 Home flags (Block printed,Bone,Brass,Crackle glaze,Debossed)
           r.home_flag),
S(4022, "custitem_ob_burn_time",                   # Burn time
        r.burn_time),
S(4024, "custitem_ob_fill_weight_g",               # Fill weight g
        r.fill_weight_g),
S(4025, "custitem_ob_fill_weight_ml",              # Fill weight ml
        r.fill_weight_ml),
S(4026, "custitem_ob_ingredients",                 # Ingredients
        r.ingredients_new),
S(4027, "custitem_ob_allergens",                   # Allergens
        r.allergens_netsuite), 
S(4028, "custitemob_directions_of_use",            # Directions of use
        r.directions_of_use),
L(4029, "custitem_ob_outdoor_use",                 # Outdoor use                     1177 Outdoor use (Yes,No)
           r.outdoor_use),
S(4030, "custitem_ob_fragrance_name",              # Fragrance name
        r.fragrance_name),
L(4031, "custitem_ob_fragrance_family_filter",     # Fragrance family filter         1179 Fragrance family filter (Floral,Fresh,Woody,Amber)
           r.fragrance_family),
S(4032, "custitem_ob_scent_notes",                 # Scent notes
        r.scent_notes),
D(4033, "custitem_ob_hanging_length_cm",           # Hanging length cm
        r.hanging_length),
M(4034, "custitem_ob_wall_hanging_fixtures",       # Wall hanging fixtures           1182 Wall hanging fixtures (Chain,Clips,D-ring,Hanging string,Hangman)
           r.wall_hanging_fixtures),
L(4035, "custitem_ob_carry_handles",               # Carry handles                   1183 Carry handles (Yes)
           r.carry_handles),
L(4036, "custitem_ob_bottle_capacity",             # Bottle capacity                 1184 Bottle capacity (1,2,3,4,5)
           r.bottle_capacity),
L(4037, "custitem_ob_glass_capacity",              # Glass capacity                  1185 Glass capacity (1,2,3,4,5)
           r.bottle_capacity), # todo glass bottle?
L(4038, "custitem_ob_nesting",                     # Nesting                         1186 Nesting (Yes)
           r.nesting),
M(4039, "custitem_ob_assembly_required",           # Assembly required               1187 Assembly required (Full assembly required,Legs to be fitted,No assembly required,Partial assembly required)
           r.assembly_required),
I(4040, "custitem_ob_fluid_capacity_ml",           # Fluid capacity ml
        r.fluid_capacity_ml),
L(4041, "custitem_ob_led_bulb",                    # Led bulb                        1189 Led bulb (Yes)
           r.ledbulb),
L(4042, "custitem_ob_recommended_bulb",            # Recommended bulb                1190 Recommended bulb (Bar Red LED Light Bulb,Calm LED Light Bulb,Crescent Moon LED Light Bulb,Festa Rainbow LED Light Bulb,Hato Heart LED Light Bulb)
           r.compatible_bulb),
L(4043, "custitem_ob_bulb_included",               # Bulb included                   1191 Bulb included (Yes,No)
           r.bulb_included),
L(4044, "custitem_ob_bulb_fitting",                # Bulb fitting                    1192 Bulb fitting (B22,E14/SES,E27/ES,G9)
           r.bulb_fitting),
L(4045, "custitem_ob_power_source",                # Power source                    1193 Power source (Batteries included,Batteries not included,Plug,Requires hardwiring)
           r.power_source),
L(4046, "custitem_ob_conversion_ring_included",    # Conversion ring included        1194 Conversion ring included (Yes,No)
           r.conversion_ring_included),
D(4047, "custitem_ob_cord_length_cm",              # Cord length cm
        r.cord_length),
L(4048, "custitem_ob_cord_colour",                 # Cord colour                     1196 Cord colour (Black & White,Grey,White,Champagne)
           r.cord_colour),
L(4049, "custitem_ob_switch_type",                 # Switch type                     1197 Switch type (Pull chain,Push button,Rocker,Toggle)
           r.switch_type),
L(4050, "custitem_ob_shade_included",              # Shade included                  1198 Shade included (Yes,No)
           r.shade_included),
L(4051, "custitem_ob_tint",                        # Tint                            1199 Tint (Copper,Gold,Green,Grey,Multi)
           r.lighting_tint),
L(4052, "custitem_ob_maximum_wattage",             # Maximum wattage                 1200 Maximum wattage (2W LED,4W LED,6W LED,7W LED,15W LED)
           r.maximum_wattage),
L(4053, "custitem_ob_hours",                       # Hours                           1201 Hours (15,000,25,000,30,000)
           r.lighting_hours),
L(4054, "custitem_ob_voltage",                     # Voltage                         1202 Voltage (220-240V,240V)
           r.voltage),
L(4055, "custitem_ob_kelvin",                      # Kelvin                          1203 Kelvin (2000K,2100K,2200K,2300K,2400K)
           r.kelvin),
L(4056, "custitem_ob_lumens",                      # Lumens                          1204 Lumens (40,80,90,100,110)
           r.lumens),
L(4057, "custitem_ob_dimmable",                    # Dimmable                        1205 Dimmable (Yes,No)
           r.dimmable),
L(4058, "custitem_ob_frame_orientation",           # Frame orientation               1206 Frame orientation (Landscape,Portrait,Portrait or landscape)
           r.frame_orientation),
L(4059, "custitem_ob_tilt",                        # Tilt                            1207 Tilt (Yes)
           r.tilt),
]



records = [
    InventoryItem(
        ################################################### ids
        externalId = f"new-MEGA-PRODUCT-{r.refnr}",
        itemId = f"new-MEGA-PRODUCT-{r.refnr}",
        #externalId = attr.scriptId,
        #itemId = attr.scriptId,

        ############################################### accounts: you can search for the number in accounts.txt
        assetAccount = RecordRef(
            # see error on externalid
            #externalId = 22010 # 22010 Current Assets: Stock& Work In Progress:...
            internalId = 1772
        ),
        cogsAccount = RecordRef(
            externalId = 61010 # 61010 Cost of Sales: Cost of Sales	
            #"internalId": 1322
        ),
        incomeAccount = RecordRef(
            externalId = 51010 # 51010 Sales : Sales : Sales
            #"internalId": 1069
        ),
        ################################################## more
        costCategory = RecordRef(
            internalId = 2,
        ),
        costEstimateType = "_averageCost",
        # cost_price??
        displayName=r.display_name,
        includeChildren = r.include_children_constant.lower() == "yes",
        #matrixType = "_parent", # permission!
        
        taxSchedule = RecordRef(
            # https://stackoverflow.com/questions/29771560/how-to-look-up-tax-schedules-using-suitetalk
            internalId = tax_map[r.tax_schedule_req_constant],
        ),
        subsidiaryList = [
            {
                "recordRef": RecordRef(
                    internalId = subsidiary_mapping[r.subsidiary_constant]
                )
             }
        ],
        #################################################### hierarchy
        hierarchyVersionsList=InventoryItemHierarchyVersionsList(
            replaceAll = False,
            inventoryItemHierarchyVersions = InventoryItemHierarchyVersions(
                isIncluded=True,
                hierarchyVersion = RecordRef(
                    internalId = 11
                    #name="OB 2023 version1"
                ),
                hierarchyNode = RecordRef(
                    externalId=merch_hierarchy_node,
                    type="merchandiseHierarchyNode"
                   # name=r.merch_hierarchy_nodes
                ),
            )
        ),
        ################################################### customFields
        customFieldList= {"customField":[
            # # coo should be M
            # S(1604, "custitem_emea_country_of_origin",         # Country of Origin               -159 None ()
            #     20), ### problem!
            # #M(4863, "custitem_ob_agent",                       # Agent                           1872 Agent (Go Sourcing,Siam Karat,Nunomoura)
            # #    r.SOMETHING),
            # #M(3064, "custitem8",                               # Brand                           1036 Brand (Branded,Own Brand)
            # #    r.brand),
            # #M(3065, "custitem9",                               # Range                           1037 Range (Botanical,Gold,Silver,Jeans,Bags)
            # #    None),
            # S(4702, "custitem25",                              # Item Description                None None ()
            #     r.name),
            # S(4818, "custitemob_supplier_name",                # Supplier Name                   -3 None ()
            #     vendor_map[r.supplier_finance]),
            # S(4816, "custitem_ob_supplier_style_number",       # Supplier Style Number           None None ()
            #     r.supplier_item_no),
            # 
            # # supplier???????
            # 
            # #M(836, "custitem_atlas_season",                    # Season                          459 Season (Fall 2015,Winter 2015,Spring 2016,Summer 2016,Fall 2016)
            # #    r.season_launch), the problem is they do not match the values!
            # ######################################## item detail
            # #M(3063, "custitem_ob_dispatch_type",               # Dispatch Type                   1035 Dispatch Type (Bulky,Generic,Large Furniture)
            # #    None),
            # #M(3066, "custitem10",                              # Factory                         1038 None ()
            # #    None),
            # S(3070, "custitem13",                              # CBM
            #     r.cbm),
            # #M(3062, "custitem6",                               # Status                          1034 FStatus (NEW,REPEAT,OLD,CURRENT,FLOW)
            # #    r.status),
            #  BooleanCustomFieldRef( # Age Restriction (Over 18)
            #     internalId = "3061",
            #     scriptId= "custitem5",
            #       value = r.agecheck_netsuite == True
            # ),
            #  BooleanCustomFieldRef( # Approved for Website
            #     internalId = "3057",
            #     scriptId= "custitem1",
            #     value = safe_lower(r.WEB) == "yes"
            # ),
            #  BooleanCustomFieldRef( # Approved for US Website
            #     internalId = "3058",
            #     scriptId= "custitem2",
            #     value = safe_lower(r.IrelandWebsite) == "yes"
            # ),
            #  BooleanCustomFieldRef( # Approved for Ireland Website
            #     internalId = "3059",
            #     scriptId= "custitem3",
            #     value = safe_lower(r.USWEBSITE) == "yes"
            # ),
            #  BooleanCustomFieldRef( # Hazmat
            #     internalId = "3060",
            #     scriptId= "custitem_ob_hazmat",
            #     value = r.hazmat != 0
            # ),
            # S(3068, "custitem11",                              # Inner
            #     r.inner),
            # S(3069, "custitem12",                              # Outer
            #     r.outer),
            # S(3071, "custitem14",                              # MOQ
            #     None),
            # #S(4763, "custitem_ob_content_ready",               # Content Attributes Ready
            # #    None),
            # #M(4764, "custitem_ob_bm_status",                   # B&M Attribute Status            1845 Item Attribute Status B&M (Ready for Content Attributes,Pending Fields but Ready for Content Attributes ,Pending B&M Attributes)
            #  #   None),
            # 
            # ############################################# matrix items
            # #M(2721, "custitemcolorcustom",                     # Colour                          975 Colour (Amber,Amethyst,Apricot,Aqua,Aubergine)
            # #    None if r.colour is None else r.colour.split(",")[0],
            # #    {'Brown/White': 'Black/White'}),
            # M(2723, "custitemsizecustomalpha",                 # XS - XXXL                       978 XS - XXXL (XS,S,M,L,XL)
            #     r.xs_xxl_matrix),
            # M(2724, "custitemsizefashion4",                    # XS/S - XL/XXL Accessories       979 XS/S - XL/XXL Accessories (XS/S,S/M,M/L,L/XL,XL/XXL)
            #     r.xs_s_xl_xxl_matrix),
            # M(2725, "custitemsizefashion5",                    # Ring Sizes - Jewellery          980 Ring Sizes - Jewellery (42,44,46,48,50)
            #     r.ring_size_matrix),
            # M(2727, "custitemsizefashion7",                    # Alphabet                        982 Alphabet (&,A,B,C,D)
            #     r.alphabet_matrix),
            # #M(4295, "custitem_ob_bedroom_textiles_matrix",     # Bedroom/Textiles                1687 Bedroom/Textiles (Double,King,45x45,30x50,60x90)
            # #    r.SOMETHING),
            # #M(4296, "custitem_ob_flavour_matrix",              # Flavour                         1684 Flavour (Linseed/Oregano/Parsley,Pumpkin Seed/Banana,Carrot Cake,Slipper/Biscuit,Lamp Post/Chips)
            # #    r.SOMETHING),
            # #M(4297, "custitem_ob_frames_matrix",               # Frames                          1685 Frames (2x2",3x4",3x3",4x4",5x7")
            # #    r.SOMETHING),
            # #M(4298, "custitem_ob_paper_size_matrix",           # Paper Size                      1686 Paper Size (A1,A2,A3,A4,A5)
            # #    r.SOMETHING),
            # #M(4299, "custitem_ob_scent_matrix",                # Scent                           1683 Scent (Amber,Avocado/Coconut,Baobab/Patchouli,Basil/Lemon,Basil/White Clay)
            # #    r.SOMETHING),
            # M(4300, "custitem22",                              # Clothing                        1688 Clothing (6,8,10,12,14)
            #     r.clothing_matrix),
            # M(4302, "custitem23",                              # Footwear                        1689 Footwear (3,3.5,4,4.5,5)
            #     r.footwear_matrix),
            # M(4747, "custitem26",                              # Stone Matrix                    1843 Stone Matrix (Amazonite,Amethyst,Aquamarine,Aventurine,Chalcedony)
            #     r.stone_matrix),
            # *webattributes
            *webattributes
        ]},  
    )
#for pos, r in fashion.iterrows()
    #for attr in webattributes
],
records = records[0] # was a tuple for some reason
records = [
    InventoryItem(
        externalId = f"new-regular-PRODUCT-{r.refnr}",
        itemId = f"new-regular-PRODUCT-{r.refnr}",
        customFieldList= {"customField":[
            *webattributes
        ]},
        assetAccount = RecordRef(
            # see error on externalid
            #externalId = 22010 # 22010 Current Assets: Stock& Work In Progress:...
            internalId = 1772
        ),
        cogsAccount = RecordRef(
            externalId = 61010 # 61010 Cost of Sales: Cost of Sales	
            #"internalId": 1322
        ),
        costCategory = RecordRef(
           internalId = 2,
        ),
        taxSchedule = RecordRef(
            # https://stackoverflow.com/questions/29771560/how-to-look-up-tax-schedules-using-suitetalk
            internalId = tax_map[r.tax_schedule_req_constant],
        ),
    )
]

#for rrr in records:
#    print(rrr.externalId)
#    api.service_proxy.add(
#        record = records,
#        _soapheaders = api.build_soap_headers()
#    )

upload_all_these_records(records, api, "updateList")

In [ ]:
records[0]

In [ ]:
import os
df = pandas.concat([
    pandas.read_csv("thread_logs/" + file)
    for file in os.listdir("thread_logs")
])
df

In [ ]:
for row in set(df.message.to_list()):
    if type(row) == float or "specified" in row:
        continue
    print(row[row.index("custitem"): row.index("due")])

In [ ]:
r = everything.iloc[0]
print(r.boxed_weight)

In [ ]:
from multithread_uploader import upload_all_these_records, response_logger
import multithread_uploader
from importlib import reload
reload(multithread_uploader)
print(len(records[:3]))
upload_all_these_records(records[:50], api, "addList")

In [ ]:
import os
pandas.concat([
    pandas.read_csv("thread_logs/" + file)
    for file in os.listdir("thread_logs")
])

### permissions
* matrix permission
* supplier_number field doesnt work, what is supplier column?
* ean
* cost price
* b__and__m_attribute_status ????? content attributes ready
* merch hierarchy
* for country and probably the cost estimate types, you need to use getSelectValue operator which is a bit long.


### names
* andre's column
* name on form
* name of custom field + script id + internal id
* name of custom list + script id + internal id

### searches
* Search
* getCustomizationId
* get
* getSelectValue